In [1]:
from pdf2image import convert_from_path
import csv
import os
import subprocess

In [2]:
# File and Directory Paths
table_dir = "extracted_tables/"
table_metadata_file = table_dir + "tables.csv"
output_dir = table_dir + "table_images/"
code_table_dir = table_dir + "table_code/"

# Max compiling time for pdflatex
MAX_COMPILING_TIME = 90

# Create output directories
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Returns a list of tables for which there does not exist an image file yet
def get_list_of_unprocessed_tables():
    # Locate already existing image files
    processed_tables = [x.replace(".png", "") for x in os.listdir(output_dir)]
    
    # Check for each table of the metadata if an image file exists
    table_metadata = open(table_metadata_file, "r", newline='', encoding='utf-8')
    spamreader = csv.reader(table_metadata, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
    list_unprocessed_papers = [row[0] for row in spamreader if row[0] not in processed_tables]
    table_metadata.close()
    
    return list_unprocessed_papers


# Removes a subset of tables from the dataset (including table_image, table_code)
def remove_from_dataset(table_subset):
    table_code_files = os.listdir(code_table_dir)
    
    for table in table_subset:
        table_code = table + ".tex"
        
        if table_code in table_code_files:
            os.remove(code_table_dir + table_code)
    
    # Remove filtered tables from metadata file
    with open(table_metadata_file, "r", newline='', encoding='utf-8') as input_file:
        with open(table_dir + "tmp.csv", "w", newline='', encoding='utf-8') as output_file:
            csv_reader = csv.reader(input_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            csv_writer = csv.writer(output_file, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            
            # Only writes rows of not deleted tables
            for row in csv_reader:
                if row[0] not in table_subset:
                    csv_writer.writerow(row)
                    
    # Replace old csv file with new csv file
    os.replace(table_dir + "tmp.csv", table_metadata_file)
    
    
# Removes compiling files (all not .png files) from output directory
def remove_compiling_files():
    for file in os.listdir(output_dir):
        if not file.endswith(".png"):
            os.remove(output_dir + file)

In [4]:
"""
Converting latex code tables to png tables
unprocessed_tables - list of table ids where table images have to be generated
amount - number of tables that should be processed
"""
def generate_table_images(unprocessed_tables, amount):
    faulty_tables = []
    for i in range(amount):          
        table_id = unprocessed_tables[i]
        print(table_id)   
        table_file = code_table_dir + table_id + ".tex"
        
        if os.path.isfile(table_file):
            try:
                # Using pdflatex to generate pdf file
                result = subprocess.run(
                    ["pdflatex", "-interaction=nonstopmode", f"-output-directory={output_dir}", table_file],
                    capture_output=True,
                    text=True,
                    timeout=MAX_COMPILING_TIME
                )
                
                # Converting pdf file to an image
                pdf_file = output_dir + table_id + ".pdf"
                images = convert_from_path(pdf_file)              
                png_file = pdf_file.replace(".pdf", ".png")            
                images[0].save(png_file, "PNG")

                # Trimming whitespace
                os.system(f"magick {png_file} -trim {png_file}")

            except Exception as e:
                faulty_tables.append(table_id)
                print(f"Error Type: {type(e).__name__}")
        else:
            print(f"{table_file} was not found.")
    
    # Removing faulty tables from dataset
    remove_from_dataset(faulty_tables)

In [5]:
unprocessed_tables = get_list_of_unprocessed_tables()
print(f"Length of unprocessed tables: {len(unprocessed_tables)}")

Length of unprocessed tables: 12542


In [6]:
# Set number of tables for which an image shall be created
number_of_tables_to_process = 12542

# Generate table images
generate_table_images(unprocessed_tables, number_of_tables_to_process)

# Remove compiling files (such as .aux and .out)
remove_compiling_files()

2202.02060_TAB_1
Error Type: PDFPageCountError
2202.02060_TAB_3
Error Type: PDFPageCountError
2203.14781_TAB_1
2205.02484_TAB_2
Error Type: PDFPageCountError
2205.02484_TAB_3
Error Type: PDFPageCountError
2205.03240_TAB_2
2205.11150_TAB_1
2205.11150_TAB_2
2206.01510_TAB_2
Error Type: PDFPageCountError
2206.01510_TAB_3
Error Type: PDFPageCountError
2206.14573_TAB_2
Error Type: PDFPageCountError
2206.14573_TAB_3
Error Type: PDFPageCountError
2206.15394_TAB_2
2206.15394_TAB_3
2207.00014_TAB_1
2207.00014_TAB_2
2207.07452_TAB_1
Error Type: PDFPageCountError
2207.10890_TAB_2
2207.13099_TAB_1
Error Type: PDFPageCountError
2207.13099_TAB_2
Error Type: PDFPageCountError
2207.13099_TAB_3
Error Type: PDFPageCountError
2207.13099_TAB_4
Error Type: PDFPageCountError
2207.14784_TAB_6
Error Type: PDFPageCountError
2207.14784_TAB_7
Error Type: PDFPageCountError
2207.14784_TAB_8
Error Type: PDFPageCountError
2208.06454_TAB_2
2209.01564_TAB_2
Error Type: PDFPageCountError
2209.01564_TAB_5
Error Type: PD

Error Type: PDFPageCountError
2302.02133_TAB_1
Error Type: PDFPageCountError
2302.02133_TAB_2
Error Type: PDFPageCountError
2302.02133_TAB_3
Error Type: PDFPageCountError
2302.02133_TAB_4
Error Type: PDFPageCountError
2302.02349_TAB_1
2302.02349_TAB_2
2302.02574_TAB_1
2302.02574_TAB_2
2302.02843_TAB_1
2302.02843_TAB_2
2302.02843_TAB_4
2202.02059_TAB_1
Error Type: PDFPageCountError
2202.02059_TAB_3
Error Type: PDFPageCountError
2203.00853_TAB_4
2203.00853_TAB_8
2204.05299_TAB_2
Error Type: PDFPageCountError
2204.05299_TAB_3
Error Type: PDFPageCountError
2204.05299_TAB_4
Error Type: PDFPageCountError
2204.10807_TAB_2
Error Type: PDFPageCountError
2204.10807_TAB_3
Error Type: PDFPageCountError
2204.12291_TAB_1
Error Type: PDFPageCountError
2204.12291_TAB_3
Error Type: PDFPageCountError
2205.01115_TAB_1
2205.01115_TAB_2
2205.01115_TAB_3
2205.10091_TAB_1
Error Type: PDFPageCountError
2205.10091_TAB_2
Error Type: PDFPageCountError
2205.10091_TAB_5
Error Type: PDFPageCountError
2205.10091_TAB

2302.05198_TAB_18
2302.05198_TAB_19
2302.05198_TAB_20
2302.05198_TAB_21
2302.05198_TAB_22
2302.05288_TAB_1
Error Type: PDFPageCountError
2201.04666_TAB_1
2201.04666_TAB_2
2202.02463_TAB_1
Error Type: PDFPageCountError
2202.10061_TAB_1
2202.10061_TAB_2
2202.10061_TAB_3
2202.10061_TAB_4
2202.12910_TAB_1
Error Type: PDFPageCountError
2206.02724_TAB_2
Error Type: PDFPageCountError
2208.00653_TAB_1
Error Type: PDFPageCountError
2208.00653_TAB_4
Error Type: PDFPageCountError
2209.11941_TAB_1
Error Type: PDFPageCountError
2209.11941_TAB_3
Error Type: PDFPageCountError
2209.15581_TAB_1
Error Type: PDFPageCountError
2209.15581_TAB_2
Error Type: PDFPageCountError
2211.12242_TAB_1
2211.12242_TAB_3
2211.12242_TAB_5
2211.12242_TAB_6
2211.12242_TAB_8
2211.12242_TAB_10
2302.05456_TAB_1
2302.05456_TAB_2
2302.05456_TAB_3
2302.05566_TAB_1
2302.05760_TAB_1
2302.05760_TAB_2
2302.06143_TAB_1
Error Type: PDFPageCountError
2302.06143_TAB_2
Error Type: PDFPageCountError
2302.06150_TAB_1
Error Type: PDFPageCou

Error Type: PDFPageCountError
2211.17153_TAB_2
Error Type: PDFPageCountError
2211.17153_TAB_3
Error Type: PDFPageCountError
2212.03240_TAB_1
Error Type: PDFPageCountError
2212.03240_TAB_2
Error Type: PDFPageCountError
2212.03240_TAB_3
Error Type: PDFPageCountError
2212.03240_TAB_4
Error Type: PDFPageCountError
2212.03240_TAB_5
Error Type: PDFPageCountError
2212.03240_TAB_6
Error Type: PDFPageCountError
2302.07050_TAB_1
2302.07879_TAB_1
2302.07879_TAB_2
2302.07924_TAB_1
Error Type: PDFPageCountError
2302.07932_TAB_1
Error Type: PDFPageCountError
2302.07932_TAB_2
Error Type: PDFPageCountError
2302.08013_TAB_1
2302.08013_TAB_2
2302.08199_TAB_1
Error Type: PDFPageCountError
2302.08389_TAB_1
Error Type: PDFPageCountError
2302.08414_TAB_1
2302.08414_TAB_2
2302.08414_TAB_3
2302.08414_TAB_4
2302.08458_TAB_1
2302.08458_TAB_2
2302.08458_TAB_3
2202.03642_TAB_1
Error Type: PDFPageCountError
2202.03642_TAB_2
Error Type: PDFPageCountError
2202.03642_TAB_3
Error Type: PDFPageCountError
2204.06819_TAB

Error Type: PDFPageCountError
2212.06770_TAB_1
Error Type: PDFPageCountError
2212.06770_TAB_2
Error Type: PDFPageCountError
2212.07169_TAB_1
Error Type: PDFPageCountError
2212.07522_TAB_2
Error Type: PDFPageCountError
2212.07522_TAB_3
Error Type: PDFPageCountError
2212.07522_TAB_4
Error Type: PDFPageCountError
2212.10126_TAB_1
2212.10126_TAB_2
2212.12039_TAB_1
Error Type: PDFPageCountError
2212.12039_TAB_2
Error Type: PDFPageCountError
2301.00578_TAB_1
Error Type: PDFPageCountError
2301.01895_TAB_1
2301.01895_TAB_2
2301.01895_TAB_3
2301.01895_TAB_4
2301.01895_TAB_5
2301.01895_TAB_6
2301.01895_TAB_7
2301.01895_TAB_8
2301.02256_TAB_1
2301.02256_TAB_2
2301.02256_TAB_3
2301.02256_TAB_4
2301.02450_TAB_1
Error Type: PDFPageCountError
2301.02450_TAB_2
Error Type: PDFPageCountError
2301.02450_TAB_3
Error Type: PDFPageCountError
2301.03311_TAB_2
Error Type: PDFPageCountError
2301.03599_TAB_1
Error Type: PDFPageCountError
2301.03752_TAB_1
Error Type: PDFPageCountError
2301.03752_TAB_3
Error Type

...

2310.07218_TAB_1
Error Type: PDFPageCountError
2310.07218_TAB_2
Error Type: PDFPageCountError
2310.07218_TAB_3
Error Type: PDFPageCountError
2310.07218_TAB_4
Error Type: PDFPageCountError
2310.07218_TAB_7
Error Type: PDFPageCountError
2310.07221_TAB_1
2310.07221_TAB_3
2310.07221_TAB_4
2310.07251_TAB_1
Error Type: PDFPageCountError
2310.07251_TAB_3
Error Type: PDFPageCountError
2310.07299_TAB_3
Error Type: PDFPageCountError
2310.07299_TAB_4
Error Type: PDFPageCountError
2310.07300_TAB_1
2310.07306_TAB_1
Error Type: PDFPageCountError
2310.07306_TAB_2
Error Type: PDFPageCountError
2310.07306_TAB_3
Error Type: PDFPageCountError
2310.07306_TAB_5
Error Type: PDFPageCountError
2310.07306_TAB_6
Error Type: PDFPageCountError
2310.07380_TAB_1
2310.07380_TAB_3
2310.07380_TAB_4
2310.07380_TAB_5
2310.07394_TAB_1
2310.07394_TAB_2
2310.07405_TAB_1
2310.07448_TAB_1
2310.07525_TAB_1
2310.07526_TAB_1
2310.07573_TAB_2
Error Type: PDFPageCountError
2310.07584_TAB_1
2310.07584_TAB_2
2310.07584_TAB_3
2310.0

Error Type: PDFPageCountError
2310.08371_TAB_4
Error Type: PDFPageCountError
2310.08737_TAB_1
2310.08757_TAB_2
2310.08778_TAB_1
2310.08778_TAB_2
2306.07402_TAB_1
Error Type: PDFPageCountError
2306.07402_TAB_2
Error Type: PDFPageCountError
2306.07402_TAB_3
Error Type: PDFPageCountError
2306.07402_TAB_6
Error Type: PDFPageCountError
2306.07402_TAB_8
Error Type: PDFPageCountError
2306.07412_TAB_1
Error Type: PDFPageCountError
2306.07415_TAB_4
Error Type: PDFPageCountError
2306.07476_TAB_2
2306.07487_TAB_1
Error Type: PDFPageCountError
2306.07487_TAB_2
Error Type: PDFPageCountError
2306.07487_TAB_3
Error Type: PDFPageCountError
2306.07489_TAB_1
2306.07489_TAB_2
2306.07489_TAB_3
2306.07499_TAB_1
Error Type: PDFPageCountError
2306.07512_TAB_2
Error Type: PDFPageCountError
2306.07512_TAB_3
Error Type: PDFPageCountError
2306.07512_TAB_5
Error Type: PDFPageCountError
2306.07512_TAB_6
Error Type: PDFPageCountError
2306.07512_TAB_7
Error Type: PDFPageCountError
2306.07512_TAB_8
Error Type: PDFPag

2304.02713_TAB_4
2304.02713_TAB_5
2304.02713_TAB_6
2304.02714_TAB_2
2304.02842_TAB_1
Error Type: PDFPageCountError
2304.02842_TAB_2
2304.03104_TAB_1
2304.03104_TAB_2
2304.03295_TAB_3
2304.03697_TAB_1
2304.03697_TAB_2
2304.03697_TAB_5
2304.03697_TAB_6
2304.03697_TAB_7
2304.03697_TAB_11
2304.03697_TAB_12
2304.03739_TAB_1
Error Type: PDFPageCountError
2304.03739_TAB_2
Error Type: PDFPageCountError
2304.03896_TAB_1
2304.03896_TAB_4
2304.04014_TAB_1
Error Type: PDFPageCountError
2304.04014_TAB_2
Error Type: PDFPageCountError
2304.04014_TAB_4
Error Type: PDFPageCountError
2304.04014_TAB_5
Error Type: PDFPageCountError
2304.04014_TAB_6
Error Type: PDFPageCountError
2304.04014_TAB_7
Error Type: PDFPageCountError
2304.04014_TAB_9
Error Type: PDFPageCountError
2304.04014_TAB_10
Error Type: PDFPageCountError
2304.04086_TAB_2
2304.04086_TAB_3
2304.04111_TAB_1
2304.04111_TAB_2
2304.04111_TAB_3
2304.04346_TAB_1
2304.04346_TAB_2
2304.04402_TAB_1
2304.04435_TAB_1
2304.04731_TAB_1
Error Type: PDFPageCo

2209.04179_TAB_1
Error Type: PDFPageCountError
2209.04179_TAB_2
Error Type: PDFPageCountError
2209.04184_TAB_3
2209.04184_TAB_4
2209.04184_TAB_5
2209.04184_TAB_6
2209.04252_TAB_2
2209.04252_TAB_3
2209.04252_TAB_4
2209.04317_TAB_1
2209.04317_TAB_3
2209.04317_TAB_4
2209.04317_TAB_6
2209.04317_TAB_7
2209.04317_TAB_8
2209.04317_TAB_9
2209.04317_TAB_10
2209.04317_TAB_11
2209.04317_TAB_12
2209.04317_TAB_13
2209.04317_TAB_14
2209.04432_TAB_2
Error Type: PDFPageCountError
2209.04432_TAB_8
Error Type: PDFPageCountError
2202.08024_TAB_1
2202.12855_TAB_4
2202.12855_TAB_5
2203.03057_TAB_2
2203.03057_TAB_3
2203.03057_TAB_5
2203.14814_TAB_1
Error Type: PDFPageCountError
2203.16027_TAB_2
Error Type: PDFPageCountError
2206.06165_TAB_1
2206.06165_TAB_3
2206.06165_TAB_4
2206.06165_TAB_5
2206.06165_TAB_6
2206.11719_TAB_1
2207.05420_TAB_1
2207.05420_TAB_2
2207.05420_TAB_3
2207.05420_TAB_4
2207.05420_TAB_5
2207.05420_TAB_6
2207.07437_TAB_1
2207.10128_TAB_2
Error Type: PDFPageCountError
2208.03408_TAB_4
Err

Error Type: PDFPageCountError
2209.06273_TAB_15
Error Type: PDFPageCountError
2209.06273_TAB_17
Error Type: PDFPageCountError
2209.06274_TAB_2
2209.06274_TAB_3
2209.06332_TAB_1
2209.06332_TAB_2
2209.06344_TAB_1
2209.06344_TAB_3
2209.06344_TAB_4
2209.06363_TAB_1
2209.06363_TAB_2
2209.06369_TAB_3
Error Type: PDFPageCountError
2209.06416_TAB_6
Error Type: PDFPageCountError
2209.06422_TAB_1
Error Type: PDFPageCountError
2209.06422_TAB_2
Error Type: PDFPageCountError
2209.06422_TAB_7
Error Type: PDFPageCountError
2209.06435_TAB_1
Error Type: PDFPageCountError
2209.06435_TAB_2
Error Type: PDFPageCountError
2209.06435_TAB_3
Error Type: PDFPageCountError
2209.06435_TAB_4
Error Type: PDFPageCountError
2209.06469_TAB_1
Error Type: PDFPageCountError
2209.06469_TAB_2
Error Type: PDFPageCountError
2209.06469_TAB_3
Error Type: PDFPageCountError
2209.06469_TAB_4
Error Type: PDFPageCountError
2209.06469_TAB_5
Error Type: PDFPageCountError
2209.06469_TAB_6
Error Type: PDFPageCountError
2209.06469_TAB_7